# calcular_tendencias(2).py

#### Con este programa determinamos, para cada punto situado en las latitudes medias del hemisferio norte, la frecuencia relativa (calculada a partir de todo el periodo temporal), así como la pendiente de la regresión lineal de la frecuencia absoluta y el p-valor de dicha regresión para cada clase. Se juntan estos resultados en tres Datarray y luego se junta todo en un Dataset que se guarda como netCDF

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
import pymannkendall as mk

#### Parámetros de entrada y número de clases (dimwt)

In [3]:
DatosEntrada="C:\\Users\\Usuario\\Desktop\\Practicas Meteogalicia\\Datos de entrada\\wtseries_interim_historical_r1i1p1_nh_1979_2005.nc"
datos_tipos=['PA', 'DANE', 'DAE', 'DASE', 'DAS', 'DASW', 'DAW', 'DANW', 'DAN', 'PDNE', 'PDE', 'PDSE', 'PDS', 'PDSW', 'PDW', 'PDNW', 'PDN', 'PC', 'DCNE', 'DCE', 'DCSE', 'DCS', 'DCSW', 'DCW', 'DCNW', 'DCN', 'U']
#periodo=slice("1979-01-01T00:00:00.000000000", "2000-01-01T18:00:00.000000000")

dimwt=27

In [4]:
datos=xr.open_dataset(DatosEntrada) #El formato es xarray.core.dataset.Dataset
datos_tiempo=datos.time #El formato es xarray.core.dataarray.DataArray
datos_tiempo=datos_tiempo.to_pandas() #El formato es un pandas.core.series.Series
datos_lon=datos.lon.values
datos_lat=datos.lat.values #Estos dos últimos son arrays

#### En esta parte filtramos la mitad del año de verano, de invierno o nos quedamos con el año completo según escojamos (en el ejemplo se filtra la mitad de año de verano). Además de forma similar a calcular_tendencias.py, creo la matriz donde voy colocar después las frecuencias de cada clase con respecto a cada año del periodo, la cual necesitare par calcular la frecuencia relativa de cada clase 

In [5]:
meses=list(np.arange(4,10))
labelmeses='AMJJAS'

periodo=pd.date_range(start=datos_tiempo[0], end=datos_tiempo[-1], freq="6h")
#El formato es pandas.core.indexes.datetimes.DatetimeIndex

locmeses=np.where(periodo.month.isin(meses))[0] #El formato es un array


periodo=periodo[locmeses] #El formato es pandas.core.indexes.datetimes.DatetimeIndex
years= np.unique(periodo.year) #El formato es array
freq_año=np.zeros((27, len(years)))

#### Defino la función de regresión lineal

In [6]:
def func(x, a, b):
    y=a*x+b
    return y

#### Creo las matrices donde pongo a continuación datos del p-valor, slope y de la frecuencia relativa para cada clase en cada punto de las latitudes medias del hemisferio norte

In [8]:
pvalor=np.zeros((len(datos_lat),len(datos_lon), dimwt))
slope=np.zeros((len(datos_lat),len(datos_lon), dimwt))
freq_relativa=np.zeros((len(datos_lat),len(datos_lon), dimwt))
for lat in range(len(datos_lat)):
    for lon in range(len(datos_lon)): 
        datos_region = datos.wtseries.sel(lon=datos_lon[lon], lat=datos_lat[lat], method='nearest') 
        datos_region=datos_region.isel(time=locmeses)
        #Este archivo es un xarray.core.dataarray.DataArray
        for i in range(len(years)):
            locaño=np.where(periodo.year.values==years[i])[0]
            datos_region_año=datos_region.isel(time=locaño) #El formato es xarray.core.dataarray.DataArray
            freq_año[:, i]=np.histogram(datos_region_año.values, bins=np.arange(1,dimwt+2))[0]
        for i in range(0, dimwt):
            x=years
            y=freq_año[i,:]
            freq_total=np.sum(y)
            freq_rel=freq_total/len(datos_region.time.values)
            freq_relativa[lat, lon, i]=freq_rel
            popt, pcov = curve_fit(func, x, y)
            a, b=popt
            MannKendall = mk.original_test(y)
            pvalor[lat,lon, i]=MannKendall[2]
            slope[lat,lon, i]=a

#### Creo DataArray de los datos de las matrices pvalor, slope y freq_relativa, uno todos los DataArrays en un Dataset y guardo todo lo anterior (DataArrays y Dataset) en archivos netCDF

In [9]:
pvalor=xr.DataArray(pvalor,dims=["lat", "lon", "Clase"], coords={"lat":list(datos_lat),"lon": list(datos_lon), "Clase": list(np.arange(1, 28))}, name="pvalor")
slope=xr.DataArray(slope,dims=["lat", "lon", "Clase"], coords={"lat":list(datos_lat),"lon": list(datos_lon), "Clase": list(np.arange(1, 28))}, name="slope")
freq_relativa=xr.DataArray(freq_relativa,dims=["lat", "lon", "Clase"], coords={"lat":list(datos_lat),"lon": list(datos_lon), "Clase": list(np.arange(1, 28))}, name="freq_relativa")
pvalor_slope_freqrelativa=xr.merge([pvalor, slope, freq_relativa])


pvalor.to_netcdf(labelmeses+'_pvalor.nc')
slope.to_netcdf(labelmeses+'_slope.nc')
freq_relativa.to_netcdf(labelmeses+'_freq_relativa.nc')
pvalor_slope_freqrelativa.to_netcdf(labelmeses+'_pvalor_slope_freqrelativa')

datos.close() 